In [ ]:
%%sh
sudo pip install spark
sudo pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58762 sha256=092ce85ba9a3c3bf1a7d0b5263bdcb9adb6a5bff9947407bace385343f4c0625
  Stored in directory: /root/.cache/pip/wheels/4e/0e/f1/164619f9920fb447d294afaae11a7715bd442ded7225953d72
Successfully built spark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=c38c98b28083233becf8966d43766090661db66056ae3a99b86e634dd1c96b3e
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions
from pyspark.sql.functions import *

In [ ]:
spark = SparkSession.builder.appName('atv_sparksql').getOrCreate()

#### 1. Crie um banco de dados no dw do Spark chamado VendasVarejo, e persista todas as tabelas neste banco de dados.

In [ ]:
# Criando o banco de dados
spark.sql('CREATE DATABASE VendasVarejo')

DataFrame[]

In [ ]:
# Visualizando todos os bancos de dados
spark.sql('SHOW DATABASES').show()

+------------+
|   namespace|
+------------+
|     default|
|vendasvarejo|
+------------+



In [ ]:
# Conectando no banco de dados
spark.sql('USE VendasVarejo')

DataFrame[]

In [ ]:
# importando a base de dados clientes
clientes = spark.read.parquet('/content/Clientes.parquet')
clientes.show(5)

+---------+--------------------+------+------+--------+
|ClienteID|             Cliente|Estado|Genero|  Status|
+---------+--------------------+------+------+--------+
|        1|Adelina Buenaventura|    RJ|     M|  Silver|
|        2|        Adelino Gago|    RJ|     M|  Silver|
|        3|     Adolfo Patrício|    PE|     M|  Silver|
|        4|    Adriana Guedelha|    RO|     F|Platinum|
|        5|       Adélio Lisboa|    SE|     M|  Silver|
+---------+--------------------+------+------+--------+
only showing top 5 rows



In [ ]:
vendas = spark.read.parquet('/content/Vendas.parquet')
vendas.show(5)

+--------+----------+---------+--------+--------+
|VendasID|VendedorID|ClienteID|    Data|   Total|
+--------+----------+---------+--------+--------+
|       1|         1|       91|1/1/2019|  8053.6|
|       2|         6|      185|1/1/2020|   150.4|
|       3|         7|       31|2/1/2020|  6087.0|
|       4|         5|       31|2/1/2019| 13828.6|
|       5|         5|       31|3/1/2018|26096.66|
+--------+----------+---------+--------+--------+
only showing top 5 rows



In [ ]:
itensVenda = spark.read.parquet('/content/ItensVendas.parquet')
itensVenda.show(5)

+---------+--------+----------+-------------+----------+--------+----------------+
|ProdutoID|VendasID|Quantidade|ValorUnitario|ValorTotal|Desconto|TotalComDesconto|
+---------+--------+----------+-------------+----------+--------+----------------+
|        2|     400|         2|       9201.0|   18402.0| 6256,68|        12145.32|
|        2|     385|         2|       9201.0|   18402.0| 5704,62|        12697.38|
|        4|     395|         2|       6892.2|   13784.4| 5100,23|         8684.17|
|        4|     367|         2|       6509.3|   13018.6| 4816,88|         8201.72|
|        2|     380|         2|      7038.77|  14077.54| 4364,04|          9713.5|
+---------+--------+----------+-------------+----------+--------+----------------+
only showing top 5 rows



In [ ]:
produtos = spark.read.parquet('/content/Produtos.parquet')
produtos.show(5)

+---------+--------------------+---------+
|ProdutoID|             Produto|    Preco|
+---------+--------------------+---------+
|        1|Bicicleta Aro 29 ...|8.852,00 |
|        2|Bicicleta Altools...|9.201,00 |
|        3|Bicicleta Gts Adv...|4.255,00 |
|        4|Bicicleta Trinc C...|7.658,00 |
|        5|Bicicleta Gometws...|2.966,00 |
+---------+--------------------+---------+
only showing top 5 rows



In [ ]:
vendedores = spark.read.parquet('/content/Vendedores.parquet')
vendedores.show()

+----------+----------------+
|VendedorID|        Vendedor|
+----------+----------------+
|         1|    Armando Lago|
|         2|Capitolino Bahía|
|         3|   Daniel Pirajá|
|         4| Godo Capiperibe|
|         5|  Hélio Liberato|
|         6|   Iberê Lacerda|
|         7|Jéssica Castelão|
|         8| Napoleão Méndez|
|         9|    Simão Rivero|
|        10|  Tobias Furtado|
+----------+----------------+



In [ ]:
# Transformando os df em tabelas
clientes.write.saveAsTable('clientes')
vendas.write.saveAsTable('vendas')
itensVenda.write.saveAsTable('itensVenda')
produtos.write.saveAsTable('produtos')
vendedores.write.saveAsTable('vendedores')


In [ ]:
spark.sql('show tables').show()

+------------+----------+-----------+
|   namespace| tableName|isTemporary|
+------------+----------+-----------+
|vendasvarejo|  clientes|      false|
|vendasvarejo|itensvenda|      false|
|vendasvarejo|  produtos|      false|
|vendasvarejo|    vendas|      false|
|vendasvarejo|vendedores|      false|
+------------+----------+-----------+



#### 2. Crie uma consulta que mostre de cada item vendido:<br>
<ul>
  <li>Nome do Cliente</li>
  <li>Data da Venda</li>
  <li>Produto</li>
  <li>Vendedor</li>
  <li>Valor Total do item</li>
<ul>

In [ ]:
spark.sql("""
          SELECT cli.Cliente,
                vd.Data,
                prod.Produto,
                vend.Vendedor,
                iv.ValorTotal
          FROM Vendas AS vd
          INNER JOIN Clientes AS cli
            ON (vd.ClienteID = cli.ClienteID)
          INNER JOIN ItensVenda AS iv
            ON (vd.VendasID = iv.VendasID)
          INNER JOIN Produtos AS prod
            ON (iv.ProdutoID = prod.ProdutoID)
          INNER JOIN Vendedores AS vend
            ON (vd.VendedorID = vend.VendedorID)  
         """).show()

+-----------------+--------+--------------------+----------------+----------+
|          Cliente|    Data|             Produto|        Vendedor|ValorTotal|
+-----------------+--------+--------------------+----------------+----------+
|   Cosme Zambujal|1/1/2019|Bicicleta Altools...|    Armando Lago|   7820.85|
|   Cosme Zambujal|1/1/2019|Bermuda Predactor...|    Armando Lago|     97.75|
|   Cosme Zambujal|1/1/2019|Camiseta Predacto...|    Armando Lago|     135.0|
|Gertrudes Hidalgo|1/1/2020|Luva De Ciclismo ...|   Iberê Lacerda|     150.4|
| Antão Corte-Real|2/1/2020|Capacete Gometws ...|Jéssica Castelão|     155.0|
| Antão Corte-Real|2/1/2020|Bicicleta Gometws...|Jéssica Castelão|    5932.0|
| Antão Corte-Real|2/1/2019|Bicicleta Altools...|  Hélio Liberato|   7820.85|
| Antão Corte-Real|2/1/2019|Bermuda Predactor...|  Hélio Liberato|     97.75|
| Antão Corte-Real|2/1/2019|Bicicleta Gometws...|  Hélio Liberato|    5910.0|
| Antão Corte-Real|3/1/2018|Bicicleta Gometws...|  Hélio Liberat